In [1]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.optim import Adam

import tqdm
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import datasetclass
from models import network


In [2]:
'''
Codes Below were used to do a final preprocessing and save the final ready data in final_ready
'''

'\nCodes Below were used to do a final preprocessing and save the final ready data in final_ready\n'

```python
df = pd.read_csv("./preprocessing/final_data.csv", sep=';').drop(['Unnamed: 0', 'alias', 'descriptionProcessed'], axis=1)
encoder = OneHotEncoder()
weekdays = df['weekday'].values.reshape(-1, 1)
encoder.fit(weekdays)
one_hot_encoded = encoder.transform(weekdays).toarray().astype(np.int8)
df.drop(['weekday'], axis=1, inplace=True)
encoded_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['weekday']))
encoded_df = pd.concat([df, encoded_df], axis=1)
encoded_df['multipleImage'] = encoded_df['multipleImage'].astype(np.int8)

print(len(encoded_df.keys()))
encoded_df.describe()
```

```python
path = './preprocessing/final_data_ready.csv'
encoded_df.to_csv(path = './preprocessing/final_data_ready.csv')
```

# Training

In [3]:
path = './preprocessing/final_data_ready.csv'
#device = ('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

dataset_train = datasetclass.InstagramUserData(path, device, train=True)
dataset_test = datasetclass.InstagramUserData(path, device, train=False)
dataset_train.__len__(), dataset_test.__len__()

(5682, 632)

In [4]:
# Create Dataloader
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=16, shuffle=True)
# Test Dataloader
some_data, some_label = next(iter(train_loader))
in_dim = some_data.shape[1]
some_data.shape, some_label.shape 

(torch.Size([32, 514, 1]), torch.Size([32]))

In [5]:
# Load the neural network
print(in_dim)
model = network.LikeCategoryPredictor(in_dim=in_dim).to(device=device)
print(model)

514
LikeCategoryPredictor(
  (feature): Sequential(
    (0): Conv1d(514, 256, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv1d(64, 32, kernel_size=(1,), stride=(1,))
  )
  (linear): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32, out_features=10, bias=True)
  )
  (softmax): Softmax(dim=1)
)


In [6]:
# First Check if everything is compatible then:
some_data = some_data.to(device)
out = model(some_data)
out.shape
# Testing Done!

torch.Size([32, 10])

In [7]:
# Training
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=.0005)

def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, batch in enumerate(tqdm.tqdm(train_loader)):
        # Every data instance is an input + label pair
        x, y = batch
        x, y = x.to(device), y.to(device)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(x)

        # Compute the loss and its gradients
        loss = criterion(outputs, y.to(torch.long))
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            #tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss


In [ ]:
epoch_number = 0

In [17]:

Epochs = 10

for epoch in range(Epochs):
    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(test_loader):
            vinputs, vlabels = vdata
            vinputs, vlabels = vinputs.to(device), vlabels.to(device)
            voutputs = model(vinputs)
            vloss = criterion(voutputs, vlabels.to(torch.long))
            running_vloss += vloss

    print(f"Running loss: {running_vloss}")
    epoch_number += 1

100%|██████████| 178/178 [00:17<00:00, 10.06it/s]


Running loss: 70.15589141845703


100%|██████████| 178/178 [00:24<00:00,  7.20it/s]


Running loss: 63.21730422973633


100%|██████████| 178/178 [00:25<00:00,  6.88it/s]


...

Running loss: 64.88211059570312


100%|██████████| 178/178 [00:11<00:00, 15.07it/s]


Running loss: 68.15251922607422


100%|██████████| 178/178 [00:12<00:00, 14.38it/s]


Running loss: 63.698238372802734


In [18]:
model.eval()
with torch.no_grad():
    some_output = model(some_data)
some_output.shape

torch.Size([32, 10])

In [19]:
torch.argmax(some_output, dim=1)

tensor([7, 4, 1, 2, 5, 4, 7, 2, 5, 8, 3, 0, 4, 7, 5, 8, 8, 5, 8, 5, 9, 9, 9, 2,
        7, 2, 6, 6, 3, 5, 2, 3])

In [20]:
some_label

tensor([7., 5., 0., 2., 5., 3., 7., 1., 4., 7., 2., 0., 4., 6., 6., 9., 6., 5.,
        6., 5., 8., 9., 8., 2., 8., 2., 6., 6., 1., 5., 0., 3.])

In [21]:
criterion(some_output, some_label.to(torch.long))

tensor(1.4550)

In [22]:
def calculate_accuracy(predicted_labels, true_labels):
    total_samples = len(true_labels)
    correct_predictions = (predicted_labels == true_labels).sum().item()
    accuracy = correct_predictions / total_samples
    return accuracy

print(calculate_accuracy(torch.argmax(some_output, dim=1), some_label)*100, '%')

46.875 %
